### Pagecontent + metadata

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json

class McDonaldsMenu:
    def __init__(self):
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"
        }
        self.menu_dict = {}  # 메뉴 이름과 설명을 저장할 딕셔너리
        self.product_status_dict = {}  # 신제품 여부를 저장할 딕셔너리
        self.menu_prices = {}
        self.all_menus = []  # 모든 메뉴 정보를 저장할 리스트
        self.size_keywords = ['Medium', 'Large', 'Small']  # 크기 관련 키워드 (맥도날드 홈페이지에서만 제거)
        self.menu_names_set = set()  # 중복 제거를 위한 메뉴 이름 저장 집합
        self.fries_price = 3000  # 후렌치후라이 가격 저장
        self.cola_price = 2600  # 코카콜라 가격 저장

        self.set_menu_price_dic = {
            "맥크리스피스리라차마요": "4900",
            "베토디스리라차마요": "3900",
            "쿼터파운더치즈": "3300",
            "더블쿼터파운더치즈": "4800",
            "맥스파이시상하이버거": "2800",
            "토마토치즈비프버거": "1300",
            "빅맥": "2900",
            "트리플치즈버거": "2900",
            "맥크리스피디럭스버거": "3600",
            "맥크리스피클래식버거": "3100",
            "1955버거": "3500",
            "맥치킨모짜렐라": "3000",
            "맥치킨": "1100",
            "더블불고기버거": "1900",
            "불고기버거": "1000",
            "슈슈버거": "1700",
            "슈비버거": "3500",
            "베이컨토마토디럭스": "3400",
            "치즈버거": "700",
            "더블치즈버거": "1700"
        }

    def extract_new_product_info(self, soup):
        # 신제품 여부 확인 (이미지에 '신제품'이라는 alt 속성이 있는 경우)
        visual_div = soup.find('div', class_='visual')
        
        if visual_div:
            img_tag = visual_div.find('img', alt=re.compile(r'신제품'))
            is_new = "신제품" if img_tag else None
        else:
            is_new = None
        
        return is_new

    def clean_size_info(self, menu_name):
        # 크기 관련 단어를 제거 (맥도날드 홈페이지에서만 사용)
        for size in self.size_keywords:
            menu_name = menu_name.replace(size, '').strip()
        return menu_name

    def remove_piece_info(self, menu_name):
        # 메뉴 이름에서 '2조각', '4조각', '6조각' 등 숫자와 '조각' 정보를 제거
        return re.sub(r'\d+조각', '', menu_name).strip()

    def fetch_menu_data(self, sub_category_seq, page_num):
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }
        try:
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()
            return response.json().get('list', [])
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")
            return []

    def fetch_menu_detail(self, menu_code, sub_category_seq):
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            menu_name_tag = soup.find('h2', class_='ko')  # 메뉴 이름
            menu_desc_tag = soup.find('div', class_='desc')  # 메뉴 설명

            # 메뉴 이름과 설명을 가져오기
            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            menu_name = re.sub(r'[™®]', '', menu_name)  # 메뉴 이름에서 특수 기호 제거
            menu_name = re.sub(r'-', '', menu_name)  # 메뉴 이름에서 하이픈(-) 제거

            clean_menu_name = self.clean_size_info(menu_name)

            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)  # 메뉴 설명에서 특수 기호 제거
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            clean_menu_name_no_spaces = re.sub(r'\s+', '', clean_menu_name)
            self.menu_dict[clean_menu_name_no_spaces] = menu_desc

            # 신제품 여부 확인 및 저장
            is_new = self.extract_new_product_info(soup)
            self.product_status_dict[clean_menu_name_no_spaces] = is_new  # 신제품 여부 저장

            # 결과 딕셔너리 구성
            result = {
                "menu_name": clean_menu_name,
                "menu_desc": menu_desc
            }

            # 신제품 여부를 추가 (있는 경우)
            if is_new:
                result["product_status"] = is_new

            return result

        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    def get_menu_list(self):
        # 각 카테고리별로 메뉴 데이터를 가져와 딕셔너리로 저장
        for sub_category_seq in [1, 7, 8, 9, 10]:  # 카테고리 번호 지정
            for page_num in range(1, 6):  # 각 카테고리에서 5 페이지까지 데이터 가져오기
                menu_data = self.fetch_menu_data(sub_category_seq, page_num)
                if not menu_data:
                    break  # 더 이상 데이터가 없으면 루프 중단
                
                for item in menu_data:
                    menu_code = item.get('seq', '')  # 메뉴 코드 가져오기
                    menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)  # 상세 정보 가져오기

        return self.menu_dict

    def fetch_price_data(self):
        for cat_id in range(11, 16):
            if cat_id == 12:
                continue
            url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            script_content = soup.find_all('script', string=re.compile('ecommerce'))

            if not script_content:
                print(f"catId {cat_id}: 데이터를 포함한 스크립트를 찾지 못했습니다.")
            else:
                name_price_pattern = re.compile(r"'name'\s*:\s*\"(.*?)\".*?'price'\s*:\s*'(\d+)", re.DOTALL)

                for script in script_content:
                    matches = name_price_pattern.findall(script.string)
                    if matches:
                        for match in matches:
                            name, price = match
                            clean_name = self.clean_text(name)  # 특수 기호 제거
                            clean_name_no_spaces = re.sub(r'\s+', '', clean_name)  # 공백 제거
                            
                            # 후렌치후라이 및 코카콜라 가격 저장
                            if '후렌치후라이' in clean_name:
                                self.fries_price = int(price)
                            elif '콜라' in clean_name or '코카콜라' in clean_name:
                                self.cola_price = int(price)

                            # 중복 검사
                            if clean_name_no_spaces not in self.menu_names_set:
                                self.menu_prices[clean_name_no_spaces] = price
                                self.menu_names_set.add(clean_name_no_spaces)  # 중복 방지를 위해 메뉴 이름을 집합에 추가

                                # 메뉴 이름과 가격 출력
                                print(f"메뉴 이름: {clean_name} | 가격: {price}원")

                                # 카테고리 정보 추출
                                category = self.extract_category(script.string)

                                # 영양 정보 및 알레르기 정보 추출
                                menu_info = self.fetch_nutritional_and_allergen_info(cat_id, clean_name_no_spaces, category)
                                if menu_info:
                                    # 메뉴 이름으로 딕셔너리의 설명을 업데이트
                                    if clean_name_no_spaces in self.menu_dict:
                                        menu_info["page_content"]["description"] = self.menu_dict[clean_name_no_spaces]

                                    # 신제품 여부 추가
                                    is_new = self.product_status_dict.get(clean_name_no_spaces, "")
                                    menu_info["page_content"]["product_status"] = is_new if is_new else "-"

                                    # 추가로 세트 가격 정보 구성 (카테고리가 "버거 & 세트"인 경우)
                                    if category == '버거 & 세트':
                                        set_price = self.calculate_set_price(clean_name_no_spaces)
                                        menu_info["page_content"]["set_price"] = f"{set_price}"

                                    # 가격을 "원" 단위로 저장
                                    menu_info["page_content"]["price"] = f"{int(price)}"

                                    # 원래 메뉴 이름으로 복원
                                    menu_info["page_content"]["name"] = self.restore_piece_info(clean_name)
                                    
                                    self.all_menus.append(menu_info)  # 정보를 리스트에 추가

        # JSON 형태로 저장
        self.save_menus_to_file()  # 파일에 저장
        return json.dumps(self.all_menus, ensure_ascii=False, indent=4)


    def restore_piece_info(self, menu_name):
        # 원래의 메뉴 이름을 복원 (조각 정보를 다시 포함)
        match = re.search(r'(\d+)조각', menu_name)  # 숫자를 포함한 조각 정보 추출
        if "맥윙" in menu_name:
            if match:
                pieces = match.group(1)  # 조각 숫자 추출
                return f"맥윙 {pieces}조각"  # 원래 메뉴 이름 복원
            else:
                return "맥윙 2조각"  # 기본적으로 2조각으로 복원
        elif "골든모짜렐라" in menu_name:
            return "골든 모짜렐라 치즈스틱 4조각"
        # 추가적인 메뉴에 대해서는 조건을 더 추가할 수 있음
        return menu_name

    def calculate_set_price(self, burger_name):
        # 공백을 제거한 버거 이름으로 비교
        burger_name_no_spaces = re.sub(r'\s+', '', burger_name)
        
        # set_menu_price_dic에서 공백을 제거한 버거 이름을 검색
        if burger_name_no_spaces in self.set_menu_price_dic:
            set_price = self.set_menu_price_dic[burger_name_no_spaces]
            return int(set_price)
        else:
            # 버거 이름이 딕셔너리에 없는 경우 기본 세트 가격을 계산
            fries_price = self.fries_price
            cola_price = self.cola_price
            burger_price = int(self.menu_prices.get(burger_name, 0))

            set_price = fries_price + cola_price + burger_price
            discounted_set_price = int((set_price * 0.75) // 100 * 100)
            return discounted_set_price

    def fetch_nutritional_and_allergen_info(self, cat_id, menu_name, category):
        url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        product_info = soup.find_all('div', class_='product-info')
        for product in product_info:
            product_name_tag = product.find('h4')
            if product_name_tag:
                product_name = self.clean_text(product_name_tag.get_text(strip=True))  # 특수 기호 제거
                product_name_no_spaces = re.sub(r'\s+', '', product_name)  # 공백 제거

                # 숫자를 제거한 후 메뉴명을 비교하도록 수정
                menu_name_no_numbers = self.remove_piece_info(menu_name)  # 메뉴명에서 숫자 제거
                product_name_no_numbers = self.remove_piece_info(product_name_no_spaces)  # 추출된 메뉴명에서 숫자 제거

                if product_name_no_numbers == menu_name_no_numbers:  # 숫자를 제거한 메뉴명으로 비교
                    price = product.find('span', class_='starting-price').get_text(strip=True)

                    # 영양 정보
                    nutrition_info = {}
                    nutrition_link = product.find('div', class_='product-nutritional-info')
                    if nutrition_link:
                        details = nutrition_link.find('div', class_='popover-wrapper')
                        if details:
                            nutrition_text = details.get_text(strip=True)
                            nutrition_info['중량(g)'] = self.extract_nutrition_value(nutrition_text, '중량')
                            nutrition_info['열량'] = self.extract_nutrition_value(nutrition_text, '열량')
                            nutrition_info['당'] = self.extract_nutrition_value(nutrition_text, '당')
                            nutrition_info['나트륨'] = self.extract_nutrition_value(nutrition_text, '나트륨')
                            nutrition_info['카페인'] = '-'  # 카페인 정보가 없는 경우 기본값 설정

                    # 알레르기 정보
                    allergy_info = {}
                    allergy_link = product.find('div', class_='product-allergen-info')
                    if allergy_link:
                        details = allergy_link.find('div', class_='popover-wrapper')
                        if details:
                            title = details.find('h4').get_text(strip=True)
                            allergen_details = details.get_text(strip=True)
                            allergy_info['menu_name'] = title
                            allergy_info['details'] = allergen_details

                            # 원산지 정보 추출
                            origin_info = self.extract_origin_info(allergen_details)
                            origin_info = self.clean_origin_info(origin_info)  # 원산지 정보에서 괄호 안 제거

                    # 결과 반환
                    return {
                        "page_content": {
                            "category": category,  # 카테고리 정보
                            "name": product_name,  # 원래 이름 유지
                            "product_status": self.extract_new_product_info(soup),  # 신제품 여부 추가
                            "description": self.menu_dict.get(menu_name_no_numbers, ""),  # 설명을 menu_dict에서 가져옴
                            "price": price,
                            "set_price": "",  # 나중에 추가할 세트 가격 필드
                            "nutrition": nutrition_info,  # 영양 정보 필드
                            "origin_info": origin_info,  # 원산지 정보 포함
                        }
                    }

        print(f"'{menu_name}'에 대한 정보를 찾을 수 없습니다.")
        return None

    def extract_category(self, script_string):
        impressions_pattern = re.compile(r'\'category\'\s*:\s*\"(.*?)\"')
        matches = impressions_pattern.findall(script_string)
        if matches:
            return matches[0]  # 첫 번째 카테고리 반환
        return '기타'

    def extract_nutrition_value(self, nutrition_text, nutrient):
        match = re.search(r'{}[\s]*([0-9]+[g|ml|Kcal|mg]*)'.format(nutrient), nutrition_text)
        return match.group(1) if match else '-'

    def extract_origin_info(self, allergen_details):
        # 원산지 정보 추출
        match = re.search(r'/\s*(.*?)$', allergen_details)  # '/' 뒤의 원산지 정보 추출
        return match.group(1) if match else '-'  # 원산지 정보가 없을 경우 기본값 설정

    def clean_origin_info(self, origin_info):
        # 괄호와 그 안의 내용을 제거
        cleaned_origin_info = re.sub(r'\(.*?\)', '', origin_info).strip()
        return cleaned_origin_info

    def clean_text(self, text):
        # HTML 태그 제거 및 텍스트 클린업
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)  # R 기호 및 TM 기호 제거
        cleaned_text = re.sub(r'\(시럽\s?없음\)', '', cleaned_text)  # 괄호와 "시럽 없음" 문자열 제거
        cleaned_text = re.sub(r'\(디핑소스\s?추가\s?구매\)', '', cleaned_text)  # 괄호와 "디핑소스 추가 구매" 문자열 제거
        cleaned_text = re.sub(r'디핑소스\s?추가\s?구매', '', cleaned_text)  # 괄호 없는 "디핑소스 추가 구매"도 제거
        cleaned_text = re.sub(r'-', '', cleaned_text)  # "코카-콜라" 같은 문자열에서 하이픈(-) 제거
        cleaned_text = cleaned_text.strip()  # 양쪽 공백 제거
        return cleaned_text

    def save_menus_to_file(self):
        # Langchain 호환 형식으로 저장
        documents = []
        for menu in self.all_menus:
            # 각 메뉴의 page_content를 저장
            documents.append({
                "page_content": {
                    "category": menu["page_content"]["category"],  # 카테고리 정보
                    "name": menu["page_content"]["name"],  # 메뉴 이름
                    "product_status": menu["page_content"].get("product_status", ""),  # 신제품 여부
                    "description": menu["page_content"].get("description", ""),  # 상세 설명
                    "price": menu["page_content"].get("price", ""),  # 가격
                    "set_burger_price": menu["page_content"].get("set_burger_price", ""),  # 세트 가격
                    "nutrition": menu["page_content"].get("nutrition", {}),  # 영양소 정보
                    "origin_info": menu["page_content"].get("origin_info", "")  # 원산지 정보
                },
                "metadata": {
                    "category": menu["page_content"]["category"],  # 카테고리 메타데이터
                    "product_status": menu["page_content"].get("product_status", ""),  # 신제품 여부
                    "price": menu["page_content"].get("price", ""),  # 가격
                    "nutrition": menu["page_content"].get("nutrition", {}),  # 영양소 정보
                    "origin_info": menu["page_content"].get("origin_info", "")  # 원산지 정보
                }
            })

        # JSON 파일로 저장
        json_output = json.dumps(documents, ensure_ascii=False, indent=4)
        with open("/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1017_metadata.json", "w", encoding="utf-8") as f:
            f.write(json_output)

        print("메뉴 정보를 json 파일에 저장했습니다.")

# McDonald's 메뉴 데이터를 수집하여 JSON 형식으로 저장 및 출력
menu = McDonaldsMenu()
menu.get_menu_list()  # 메뉴 이름 및 설명 딕셔너리 채우기
menu_data = menu.fetch_price_data()  


### Pagecontent만

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json

class McDonaldsMenu:
    def __init__(self):
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"
        }
        self.menu_dict = {}  # 메뉴 이름과 설명을 저장할 딕셔너리
        self.product_status_dict = {}  # 신제품 여부를 저장할 딕셔너리
        self.menu_prices = {}
        self.all_menus = []  # 모든 메뉴 정보를 저장할 리스트
        self.size_keywords = ['Medium', 'Large', 'Small']  # 크기 관련 키워드 (맥도날드 홈페이지에서만 제거)
        self.menu_names_set = set()  # 중복 제거를 위한 메뉴 이름 저장 집합
        self.fries_price = 3000  # 후렌치후라이 가격 저장
        self.cola_price = 2600  # 코카콜라 가격 저장

        self.set_menu_price_dic = {
            "맥크리스피스리라차마요": "4900",
            "베토디스리라차마요": "3900",
            "쿼터파운더치즈": "3300",
            "더블쿼터파운더치즈": "4800",
            "맥스파이시상하이버거": "2800",
            "토마토치즈비프버거": "1300",
            "빅맥": "2900",
            "트리플치즈버거": "2900",
            "맥크리스피디럭스버거": "3600",
            "맥크리스피클래식버거": "3100",
            "1955버거": "3500",
            "맥치킨모짜렐라": "3000",
            "맥치킨": "1100",
            "더블불고기버거": "1900",
            "불고기버거": "1000",
            "슈슈버거": "1700",
            "슈비버거": "3500",
            "베이컨토마토디럭스": "3400",
            "치즈버거": "700",
            "더블치즈버거": "1700"
        }

    def extract_new_product_info(self, soup):
        # 신제품 여부 확인 (이미지에 '신제품'이라는 alt 속성이 있는 경우)
        visual_div = soup.find('div', class_='visual')
        
        if visual_div:
            img_tag = visual_div.find('img', alt=re.compile(r'신제품'))
            is_new = "신제품" if img_tag else None
        else:
            is_new = None
        
        return is_new

    def clean_size_info(self, menu_name):
        # 크기 관련 단어를 제거 (맥도날드 홈페이지에서만 사용)
        for size in self.size_keywords:
            menu_name = menu_name.replace(size, '').strip()
        return menu_name

    def remove_piece_info(self, menu_name):
        # 메뉴 이름에서 '2조각', '4조각', '6조각' 등 숫자와 '조각' 정보를 제거
        return re.sub(r'\d+조각', '', menu_name).strip()

    def fetch_menu_data(self, sub_category_seq, page_num):
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }
        try:
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()
            return response.json().get('list', [])
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")
            return []

    def fetch_menu_detail(self, menu_code, sub_category_seq):
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            menu_name_tag = soup.find('h2', class_='ko')  # 메뉴 이름
            menu_desc_tag = soup.find('div', class_='desc')  # 메뉴 설명

            # 메뉴 이름과 설명을 가져오기
            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            menu_name = re.sub(r'[™®]', '', menu_name)  # 메뉴 이름에서 특수 기호 제거
            menu_name = re.sub(r'-', '', menu_name)  # 메뉴 이름에서 하이픈(-) 제거

            clean_menu_name = self.clean_size_info(menu_name)

            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)  # 메뉴 설명에서 특수 기호 제거
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            clean_menu_name_no_spaces = re.sub(r'\s+', '', clean_menu_name)
            self.menu_dict[clean_menu_name_no_spaces] = menu_desc

            # 신제품 여부 확인 및 저장
            is_new = self.extract_new_product_info(soup)
            self.product_status_dict[clean_menu_name_no_spaces] = is_new  # 신제품 여부 저장

            # 결과 딕셔너리 구성
            result = {
                "menu_name": clean_menu_name,
                "menu_desc": menu_desc
            }

            # 신제품 여부를 추가 (있는 경우)
            if is_new:
                result["product_status"] = is_new

            return result

        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    def get_menu_list(self):
        # 각 카테고리별로 메뉴 데이터를 가져와 딕셔너리로 저장
        for sub_category_seq in [1, 7, 8, 9, 10]:  # 카테고리 번호 지정
            for page_num in range(1, 6):  # 각 카테고리에서 5 페이지까지 데이터 가져오기
                menu_data = self.fetch_menu_data(sub_category_seq, page_num)
                if not menu_data:
                    break  # 더 이상 데이터가 없으면 루프 중단
                
                for item in menu_data:
                    menu_code = item.get('seq', '')  # 메뉴 코드 가져오기
                    menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)  # 상세 정보 가져오기

        return self.menu_dict

    def fetch_price_data(self):
        for cat_id in range(11, 16):
            if cat_id == 12:
                continue
            url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            script_content = soup.find_all('script', string=re.compile('ecommerce'))

            if not script_content:
                print(f"catId {cat_id}: 데이터를 포함한 스크립트를 찾지 못했습니다.")
            else:
                name_price_pattern = re.compile(r"'name'\s*:\s*\"(.*?)\".*?'price'\s*:\s*'(\d+)", re.DOTALL)

                for script in script_content:
                    matches = name_price_pattern.findall(script.string)
                    if matches:
                        for match in matches:
                            name, price = match
                            clean_name = self.clean_text(name)  # 특수 기호 제거
                            clean_name_no_spaces = re.sub(r'\s+', '', clean_name)  # 공백 제거
                            
                            # 후렌치후라이 및 코카콜라 가격 저장
                            if '후렌치후라이' in clean_name:
                                self.fries_price = int(price)
                            elif '콜라' in clean_name or '코카콜라' in clean_name:
                                self.cola_price = int(price)

                            # 중복 검사
                            if clean_name_no_spaces not in self.menu_names_set:
                                self.menu_prices[clean_name_no_spaces] = price
                                self.menu_names_set.add(clean_name_no_spaces)  # 중복 방지를 위해 메뉴 이름을 집합에 추가

                                # 메뉴 이름과 가격 출력
                                print(f"메뉴 이름: {clean_name} | 가격: {price}원")

                                # 카테고리 정보 추출
                                category = self.extract_category(script.string)

                                # 영양 정보 및 알레르기 정보 추출
                                menu_info = self.fetch_nutritional_and_allergen_info(cat_id, clean_name_no_spaces, category)
                                if menu_info:
                                    # 메뉴 이름으로 딕셔너리의 설명을 업데이트
                                    if clean_name_no_spaces in self.menu_dict:
                                        menu_info["page_content"]["description"] = self.menu_dict[clean_name_no_spaces]

                                    # 신제품 여부 추가
                                    is_new = self.product_status_dict.get(clean_name_no_spaces, "")
                                    menu_info["page_content"]["product_status"] = is_new if is_new else "-"

                                    # 추가로 세트 가격 정보 구성 (카테고리가 "버거 & 세트"인 경우)
                                    if category == '버거 & 세트':
                                        set_price = self.calculate_set_price(clean_name_no_spaces)
                                        menu_info["page_content"]["set_price"] = f"{set_price}"

                                    # 가격을 "원" 단위로 저장
                                    menu_info["page_content"]["price"] = f"{int(price)}"

                                    # 원래 메뉴 이름으로 복원
                                    menu_info["page_content"]["name"] = self.restore_piece_info(clean_name)
                                    
                                    self.all_menus.append(menu_info)  # 정보를 리스트에 추가

        # JSON 형태로 저장
        self.save_menus_to_file()  # 파일에 저장
        return json.dumps(self.all_menus, ensure_ascii=False, indent=4)


    def restore_piece_info(self, menu_name):
        # 원래의 메뉴 이름을 복원 (조각 정보를 다시 포함)
        match = re.search(r'(\d+)조각', menu_name)  # 숫자를 포함한 조각 정보 추출
        if "맥윙" in menu_name:
            if match:
                pieces = match.group(1)  # 조각 숫자 추출
                return f"맥윙 {pieces}조각"  # 원래 메뉴 이름 복원
            else:
                return "맥윙 2조각"  # 기본적으로 2조각으로 복원
        elif "골든모짜렐라" in menu_name:
            return "골든 모짜렐라 치즈스틱 4조각"
        # 추가적인 메뉴에 대해서는 조건을 더 추가할 수 있음
        return menu_name

    def calculate_set_price(self, burger_name):
        # 공백을 제거한 버거 이름으로 비교
        burger_name_no_spaces = re.sub(r'\s+', '', burger_name)
        
        # set_menu_price_dic에서 공백을 제거한 버거 이름을 검색
        if burger_name_no_spaces in self.set_menu_price_dic:
            set_price = self.set_menu_price_dic[burger_name_no_spaces]
            return int(set_price)
        else:
            # 버거 이름이 딕셔너리에 없는 경우 기본 세트 가격을 계산
            fries_price = self.fries_price
            cola_price = self.cola_price
            burger_price = int(self.menu_prices.get(burger_name, 0))

            set_price = fries_price + cola_price + burger_price
            discounted_set_price = int((set_price * 0.75) // 100 * 100)
            return discounted_set_price

    def fetch_nutritional_and_allergen_info(self, cat_id, menu_name, category):
        url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        product_info = soup.find_all('div', class_='product-info')
        for product in product_info:
            product_name_tag = product.find('h4')
            if product_name_tag:
                product_name = self.clean_text(product_name_tag.get_text(strip=True))  # 특수 기호 제거
                product_name_no_spaces = re.sub(r'\s+', '', product_name)  # 공백 제거

                # 숫자를 제거한 후 메뉴명을 비교하도록 수정
                menu_name_no_numbers = self.remove_piece_info(menu_name)  # 메뉴명에서 숫자 제거
                product_name_no_numbers = self.remove_piece_info(product_name_no_spaces)  # 추출된 메뉴명에서 숫자 제거

                if product_name_no_numbers == menu_name_no_numbers:  # 숫자를 제거한 메뉴명으로 비교
                    price = product.find('span', class_='starting-price').get_text(strip=True)

                    # 영양 정보
                    nutrition_info = {}
                    nutrition_link = product.find('div', class_='product-nutritional-info')
                    if nutrition_link:
                        details = nutrition_link.find('div', class_='popover-wrapper')
                        if details:
                            nutrition_text = details.get_text(strip=True)
                            nutrition_info['중량(g)'] = self.extract_nutrition_value(nutrition_text, '중량')
                            nutrition_info['열량'] = self.extract_nutrition_value(nutrition_text, '열량')
                            nutrition_info['당'] = self.extract_nutrition_value(nutrition_text, '당')
                            nutrition_info['나트륨'] = self.extract_nutrition_value(nutrition_text, '나트륨')
                            nutrition_info['카페인'] = '-'  # 카페인 정보가 없는 경우 기본값 설정

                    # 알레르기 정보
                    allergy_info = {}
                    allergy_link = product.find('div', class_='product-allergen-info')
                    if allergy_link:
                        details = allergy_link.find('div', class_='popover-wrapper')
                        if details:
                            title = details.find('h4').get_text(strip=True)
                            allergen_details = details.get_text(strip=True)
                            allergy_info['menu_name'] = title
                            allergy_info['details'] = allergen_details

                            # 원산지 정보 추출
                            origin_info = self.extract_origin_info(allergen_details)
                            origin_info = self.clean_origin_info(origin_info)  # 원산지 정보에서 괄호 안 제거

                    # 결과 반환
                    return {
                        "page_content": {
                            "category": category,  # 카테고리 정보
                            "name": product_name,  # 원래 이름 유지
                            "product_status": self.extract_new_product_info(soup),  # 신제품 여부 추가
                            "description": self.menu_dict.get(menu_name_no_numbers, ""),  # 설명을 menu_dict에서 가져옴
                            "price": price,
                            "set_price": "",  # 나중에 추가할 세트 가격 필드
                            "nutrition": nutrition_info,  # 영양 정보 필드
                            "origin_info": origin_info,  # 원산지 정보 포함
                        }
                    }

        print(f"'{menu_name}'에 대한 정보를 찾을 수 없습니다.")
        return None

    def extract_category(self, script_string):
        impressions_pattern = re.compile(r'\'category\'\s*:\s*\"(.*?)\"')
        matches = impressions_pattern.findall(script_string)
        if matches:
            return matches[0]  # 첫 번째 카테고리 반환
        return '기타'

    def extract_nutrition_value(self, nutrition_text, nutrient):
        match = re.search(r'{}[\s]*([0-9]+[g|ml|Kcal|mg]*)'.format(nutrient), nutrition_text)
        return match.group(1) if match else '-'

    def extract_origin_info(self, allergen_details):
        # 원산지 정보 추출
        match = re.search(r'/\s*(.*?)$', allergen_details)  # '/' 뒤의 원산지 정보 추출
        return match.group(1) if match else '-'  # 원산지 정보가 없을 경우 기본값 설정

    def clean_origin_info(self, origin_info):
        # 괄호와 그 안의 내용을 제거
        cleaned_origin_info = re.sub(r'\(.*?\)', '', origin_info).strip()
        return cleaned_origin_info

    def clean_text(self, text):
        # HTML 태그 제거 및 텍스트 클린업
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)  # R 기호 및 TM 기호 제거
        cleaned_text = re.sub(r'\(시럽\s?없음\)', '', cleaned_text)  # 괄호와 "시럽 없음" 문자열 제거
        cleaned_text = re.sub(r'\(디핑소스\s?추가\s?구매\)', '', cleaned_text)  # 괄호와 "디핑소스 추가 구매" 문자열 제거
        cleaned_text = re.sub(r'디핑소스\s?추가\s?구매', '', cleaned_text)  # 괄호 없는 "디핑소스 추가 구매"도 제거
        cleaned_text = re.sub(r'-', '', cleaned_text)  # "코카-콜라" 같은 문자열에서 하이픈(-) 제거
        cleaned_text = cleaned_text.strip()  # 양쪽 공백 제거
        return cleaned_text

    def save_menus_to_file(self):
        # Langchain 호환 형식으로 저장
        documents = []
        for menu in self.all_menus:
            # 각 메뉴의 page_content를 저장
            documents.append({
                "page_content": {
                    "category": menu["page_content"]["category"],  # 카테고리 정보
                    "name": menu["page_content"]["name"],  # 메뉴 이름
                    "product_status": menu["page_content"].get("product_status", ""),  # 신제품 여부
                    "description": menu["page_content"].get("description", ""),  # 상세 설명
                    "price": menu["page_content"].get("price", ""),  # 가격
                    "set_burger_price": menu["page_content"].get("set_burger_price", ""),  # 세트 가격
                    "nutrition": menu["page_content"].get("nutrition", {}),  # 영양소 정보
                    "origin_info": menu["page_content"].get("origin_info", "")  # 원산지 정보
                },
            })

        # JSON 파일로 저장
        json_output = json.dumps(documents, ensure_ascii=False, indent=4)
        with open("/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1017.json", "w", encoding="utf-8") as f:
            f.write(json_output)

        print("메뉴 정보를 json 파일에 저장했습니다.")

# McDonald's 메뉴 데이터를 수집하여 JSON 형식으로 저장 및 출력
menu = McDonaldsMenu()
menu.get_menu_list()  # 메뉴 이름 및 설명 딕셔너리 채우기
menu_data = menu.fetch_price_data()  
